## Common Section

In [1]:
# Google Drive Operations - Only for Google Drive, Delete in Local Settings
# Reference for using R in Colab: https://towardsdatascience.com/how-to-use-r-in-google-colab-b6e02d736497

%load_ext rpy2.ipython
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/Versed Chimpanzee - AC Group")
!ls


Mounted at /content/drive
'AC Use Cases Draft.pdf'	     interim_data
 customers.csv			     Plan.gdoc
'data columns with colors.pdf'	     submission_random.csv
'Feature Importance Scores.gsheet'   transactions.csv
 geo.csv			    'Versed Chimpanzee.ipynb'
'Info - Analytics Cup 2022.pdf'


## 1. Data Preprocessing

### 1.1 Prepare Libraries

In [2]:
%%R

library(dplyr, warn.conflicts = F, quietly = T)
library(tidyverse, warn.conflicts = F, quietly = T)
library(lubridate, warn.conflicts = F, quietly = T)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ stringr 1.4.0
✔ tidyr   1.1.4     ✔ forcats 0.5.1
✔ readr   2.1.1     

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



### 1.2 Load and Inspect Transactions (transactions.csv)

In [3]:
%%R

trnsc_df = as_tibble(read_csv("transactions.csv"))

print("[INFO] Number of NA values in each column:")
for (i in 1:ncol(trnsc_df)) {
  print(paste0(names(trnsc_df)[i], ": ", sum(is.na(trnsc_df[, i])), "/", nrow(trnsc_df)))
}

trnsc_df$CUSTOMER = (substring(trnsc_df$CUSTOMER, 2, nchar(trnsc_df$CUSTOMER)-1))

Rows: 26151 Columns: 23
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (12): MO_ID, SO_ID, CUSTOMER, END_CUSTOMER, PRICE_LIST, MO_CREATED_DATE,...
dbl (11): OFFER_PRICE, SERVICE_LIST_PRICE, MATERIAL_COST, SERVICE_COST, ISIC...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
[1] "[INFO] Number of NA values in each column:"
[1] "MO_ID: 0/26151"
[1] "SO_ID: 0/26151"
[1] "CUSTOMER: 0/26151"
[1] "END_CUSTOMER: 20114/26151"
[1] "OFFER_PRICE: 0/26151"
[1] "SERVICE_LIST_PRICE: 0/26151"
[1] "MATERIAL_COST: 0/26151"
[1] "SERVICE_COST: 0/26151"
[1] "PRICE_LIST: 0/26151"
[1] "ISIC: 1675/26151"
[1] "MO_CREATED_DATE: 0/26151"
[1] "SO_CREATED_DATE: 0/26151"
[1] "TECH: 0/26151"
[1] "OFFER_TYPE: 0/26151"
[1] "BUSINESS_TYPE: 0/26151"
[1] "COSTS_PRODUCT_A: 0/26151"
[1] "COSTS_PRODUCT_B: 0/26151"
[1] "COSTS_PRODUCT_C: 0/26151"
[1] "OFFER_STATUS: 25

### 1.3 Load, Inspect and Merge Geographic Data (geo.csv)

In [4]:
%%R

geo_df = as_tibble(read_csv("geo.csv"))

print("[INFO] Number of NA values in each column:")
for (i in 1:ncol(geo_df)) {
  print(paste0(names(geo_df)[i], ": ", sum(is.na(geo_df[, i])), "/", nrow(geo_df)))
}
# Rename column COUNTRY COUNTRY_CODE, since it only contains codes like CH, FR
geo_df = rename(geo_df, COUNTRY_CODE = COUNTRY)

# Perform left join using dplyr
trnsc_geo_df = left_join(trnsc_df, geo_df, by = 'SALES_LOCATION')

# FIXME: Delete this before submission.
write_csv(x = trnsc_geo_df, file = "interim_data/trnsc_geo_df.csv")


Rows: 46 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): COUNTRY, SALES_OFFICE, SALES_BRANCH, SALES_LOCATION

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
[1] "[INFO] Number of NA values in each column:"
[1] "COUNTRY: 0/46"
[1] "SALES_OFFICE: 2/46"
[1] "SALES_BRANCH: 1/46"
[1] "SALES_LOCATION: 1/46"


### 1.4 Load, Inspect and Merge Customers Data (customers.csv)

In [5]:
%%R

cst_df = as_tibble(read_csv("customers.csv"))

print("[INFO] Number of NA values in each column:")
for (i in 1:ncol(cst_df)) {
  print(paste0(names(cst_df)[i], ": ", sum(is.na(cst_df[, i])), "/", nrow(cst_df)))
}

##### NOTE FROM TEO: I THINK THE MERGE MAY BE WRONG, I'M MARKING THE SECTION ##########
##### WHICH I THINK MAY NEED REPLACEMENT (SEE MY SECTION) #################
##### REPLACE FROM HERE ###################
# Change data type of CUSTOMER column in customers dataset
cst_df$CUSTOMER <- as.character(cst_df$CUSTOMER)

# Create IDX_CUSTOMER for trnsc_geo_df
trnsc_geo_df = mutate(trnsc_geo_df,
                      IDX_CUSTOMER = paste0(COUNTRY_CODE, "_", CUSTOMER))

# Create IDX_CUSTOMER for customers
cst_df = cst_df %>% mutate(COUNTRY_CODE = case_when(
  COUNTRY == 'Switzerland' ~ "CH",
  COUNTRY == 'France' ~ "FR"
))

cst_df = mutate(cst_df, IDX_CUSTOMER = paste0(COUNTRY_CODE, "_", CUSTOMER))

# Perform left join using dplyr
all_merged = left_join(trnsc_geo_df, cst_df, by = 'IDX_CUSTOMER')
####### TO HERE ###############################

# FIXME: Delete this before submisssion.
write_csv(x = all_merged, file = "interim_data/all_merged.csv")

Rows: 8452 Columns: 8
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (5): REV_CURRENT_YEAR, CREATION_YEAR, OWNERSHIP, COUNTRY, CURRENCY
dbl (3): CUSTOMER, REV_CURRENT_YEAR.1, REV_CURRENT_YEAR.2

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
[1] "[INFO] Number of NA values in each column:"
[1] "CUSTOMER: 0/8452"
[1] "REV_CURRENT_YEAR: 0/8452"
[1] "REV_CURRENT_YEAR.1: 0/8452"
[1] "REV_CURRENT_YEAR.2: 0/8452"
[1] "CREATION_YEAR: 0/8452"
[1] "OWNERSHIP: 0/8452"
[1] "COUNTRY: 0/8452"
[1] "CURRENCY: 0/8452"


### 1.5 Fix Basic Problems in Merged Data

In [6]:
%%R

print(paste("[INFO] Number Of Cols:", ncol(all_merged)))
print(paste("[INFO] Number Of Rows:", nrow(all_merged)))
cat("[INFO] Names Of Columns:\n", sprintf("\"%s\", ", names(all_merged)))

cat("\n\n----------------------------------------------------\n\n")

print("[INFO] Glimpse:")
glimpse(all_merged)

cat("\n----------------------------------------------------\n\n")

print("[INFO] First 20 Rows:")
print(head(all_merged, n = 20))

[1] "[INFO] Number Of Cols: 36"
[1] "[INFO] Number Of Rows: 26151"
[INFO] Names Of Columns:
 "MO_ID",  "SO_ID",  "CUSTOMER.x",  "END_CUSTOMER",  "OFFER_PRICE",  "SERVICE_LIST_PRICE",  "MATERIAL_COST",  "SERVICE_COST",  "PRICE_LIST",  "ISIC",  "MO_CREATED_DATE",  "SO_CREATED_DATE",  "TECH",  "OFFER_TYPE",  "BUSINESS_TYPE",  "COSTS_PRODUCT_A",  "COSTS_PRODUCT_B",  "COSTS_PRODUCT_C",  "OFFER_STATUS",  "COSTS_PRODUCT_D",  "COSTS_PRODUCT_E",  "SALES_LOCATION",  "TEST_SET_ID",  "COUNTRY_CODE.x",  "SALES_OFFICE",  "SALES_BRANCH",  "IDX_CUSTOMER",  "CUSTOMER.y",  "REV_CURRENT_YEAR",  "REV_CURRENT_YEAR.1",  "REV_CURRENT_YEAR.2",  "CREATION_YEAR",  "OWNERSHIP",  "COUNTRY",  "CURRENCY",  "COUNTRY_CODE.y", 

----------------------------------------------------

[1] "[INFO] Glimpse:"
Rows: 26,151
Columns: 36
$ MO_ID              <chr> "a050N000013fnfrQAA", "a050N000013fgL1QAI", "a050N0…
$ SO_ID              <chr> "a030N00001EochoQAB", "a030N00001EociNQAR", "a030N0…
$ CUSTOMER.x         <chr> "1", "

In [7]:
%%R

# Feature Elimination & Fix
all_merged = select(all_merged, -c(COUNTRY_CODE.y, CUSTOMER.y, COUNTRY, MO_ID, SO_ID))

# Feature Renaming
all_merged = rename(all_merged, COUNTRY_CODE = COUNTRY_CODE.x)
all_merged = rename(all_merged, CUSTOMER = CUSTOMER.x)

In [8]:
%%R

print("[INFO] Number of NA values in each column:")
for (i in 1:ncol(all_merged)) { # for-loop over columns
  print(paste0(names(all_merged)[i], ": ", sum(is.na(all_merged[, i])), "/", nrow(all_merged)))
}

[1] "[INFO] Number of NA values in each column:"
[1] "CUSTOMER: 0/26151"
[1] "END_CUSTOMER: 20114/26151"
[1] "OFFER_PRICE: 0/26151"
[1] "SERVICE_LIST_PRICE: 0/26151"
[1] "MATERIAL_COST: 0/26151"
[1] "SERVICE_COST: 0/26151"
[1] "PRICE_LIST: 0/26151"
[1] "ISIC: 1675/26151"
[1] "MO_CREATED_DATE: 0/26151"
[1] "SO_CREATED_DATE: 0/26151"
[1] "TECH: 0/26151"
[1] "OFFER_TYPE: 0/26151"
[1] "BUSINESS_TYPE: 0/26151"
[1] "COSTS_PRODUCT_A: 0/26151"
[1] "COSTS_PRODUCT_B: 0/26151"
[1] "COSTS_PRODUCT_C: 0/26151"
[1] "OFFER_STATUS: 2576/26151"
[1] "COSTS_PRODUCT_D: 0/26151"
[1] "COSTS_PRODUCT_E: 0/26151"
[1] "SALES_LOCATION: 37/26151"
[1] "TEST_SET_ID: 23575/26151"
[1] "COUNTRY_CODE: 0/26151"
[1] "SALES_OFFICE: 38/26151"
[1] "SALES_BRANCH: 37/26151"
[1] "IDX_CUSTOMER: 0/26151"
[1] "REV_CURRENT_YEAR: 2885/26151"
[1] "REV_CURRENT_YEAR.1: 2885/26151"
[1] "REV_CURRENT_YEAR.2: 2885/26151"
[1] "CREATION_YEAR: 2885/26151"
[1] "OWNERSHIP: 2885/26151"
[1] "CURRENCY: 2885/26151"


In [9]:
%%R

print("[INFO] Unique values in each column:")
for (i in 1:ncol(all_merged)) {
  count_unq_vals = count(unique(all_merged[, i]))
  if (count_unq_vals <20) {
    print(paste0(names(all_merged)[i], ": ", unique(all_merged[, i])))
    print("---------------")
  }
}

# FIXME: Delete this before submission.
write_csv(x = all_merged, file = "interim_data/all_merged_2.csv")

[1] "[INFO] Unique values in each column:"
[1] "PRICE_LIST: c(\"SFT Standard\", \"CMT Installer\", \"CMT End Customer\", \"Tarif public\")"
[1] "---------------"
[1] "TECH: c(\"S\", \"C\", \"F\", \"BP\", \"FP\", \"EPS\", \"E\")"
[1] "---------------"
[1] "BUSINESS_TYPE: c(\"E\", \"N\", \"M\", \"C\", \"T\", \"Exp\", \"New\", \"Mig\", \"S\", \"F\", \"R\")"
[1] "---------------"
[1] "OFFER_STATUS: c(\"LOsT\", \"Lost\", \"WIN\", \"Win\", NA, \"Won\", \"LOST\", \"Lose\", \"WON\")"
[1] "---------------"
[1] "COUNTRY_CODE: c(\"CH\", \"FR\")"
[1] "---------------"
[1] "SALES_BRANCH: c(\"Branch Central\", \"Branch East\", \"Branch West\", \"EPS CH\", NA, \"Grand Paris\", \"Sud Ouest\", \"Nord FR\", \"Ouest\", \"Centre-Est\", \"Grand Est\", \"Sud-Est\", \"Enterprise Business France\", \"SI\")"
[1] "---------------"
[1] "OWNERSHIP: c(\"Privately Owned/Publicly Traded\", NA, \"Governmental\", \"No information\", \"Individual Person\")"
[1] "---------------"
[1] "CURRENCY: c(\"Chinese Yuan\", NA, \

In [10]:
%%R

merged_df = all_merged
write_csv(x = all_merged, file = "interim_data/merged_df.csv")

### 1.6 Split Data into Labeled and Unlabeled (Test) Sets

In [11]:
%%R

## TODO: YOU CAN SPLIT AFTERWARDS
labeled_data = all_merged[is.na(all_merged$TEST_SET_ID),]
test_data = all_merged[!is.na(all_merged$TEST_SET_ID),]

# FIXME: Delete this before submission.
write_csv(x = test_data, file = "interim_data/labeled_set.csv")
write_csv(x = test_data, file = "interim_data/test_set.csv")

labeled_data %>% nrow()
test_data %>% nrow()

[1] 2576


## 2. Feature Elimination and Manipulation

In [12]:
%%R

df = data.frame(merged_df)

### 2.1 Delete Unnecessary Columns

In [13]:
%%R

# Delete columns: MO_CREATED_DATE, SALES_BRANCH, REV_CURRENT_YEAR
df = select (df,-c(MO_CREATED_DATE,SALES_BRANCH,REV_CURRENT_YEAR))

names(df)

 [1] "CUSTOMER"           "END_CUSTOMER"       "OFFER_PRICE"       
 [4] "SERVICE_LIST_PRICE" "MATERIAL_COST"      "SERVICE_COST"      
 [7] "PRICE_LIST"         "ISIC"               "SO_CREATED_DATE"   
[10] "TECH"               "OFFER_TYPE"         "BUSINESS_TYPE"     
[13] "COSTS_PRODUCT_A"    "COSTS_PRODUCT_B"    "COSTS_PRODUCT_C"   
[16] "OFFER_STATUS"       "COSTS_PRODUCT_D"    "COSTS_PRODUCT_E"   
[19] "SALES_LOCATION"     "TEST_SET_ID"        "COUNTRY_CODE"      
[22] "SALES_OFFICE"       "IDX_CUSTOMER"       "REV_CURRENT_YEAR.1"
[25] "REV_CURRENT_YEAR.2" "CREATION_YEAR"      "OWNERSHIP"         
[28] "CURRENCY"          


### 2.2 Process Binary Features

In [14]:
# Processed Features in 2.2:
#  * OFFER_STATUS         [Modified]
#  * END_CUSTOMER         [Deleted]
#  * HAS_END_CUSTOMER     [Created]
#  * ISIC                 [Deleted]
#  * HAS_ISIC             [Created]
#  * HAS_COSTS_PRODUCT_A  [Created]
#  * HAS_COSTS_PRODUCT_B  [Created]
#  * HAS_COSTS_PRODUCT_C  [Created]
#  * HAS_COSTS_PRODUCT_D  [Created]
#  * HAS_COSTS_PRODUCT_E  [Created]
#  * COUNTRY_CODE         [Deleted]
#  * IS_COUNTRY_CODE_CH   [Created]

In [15]:
%%R

# Manipulate "OFFER_STATUS" Feature: Replace string values with binary values
df = df %>%
  mutate(
    OFFER_STATUS = case_when(
      OFFER_STATUS %in% c("WIN","Win","Won","WON") ~ "1",
      OFFER_STATUS %in% c("LOsT","Lost","LOST","Lose") ~ "0",
    )
  )

In [16]:
%%R

# Manipulate "END_CUSTOMER" Feature: Convert to HAS_END_CUSTOMER
df = df %>%
  mutate(
    HAS_END_CUSTOMER = case_when(
      END_CUSTOMER %in% c(NA,"No") ~ 0,
      TRUE ~ 1 # Includes numbers and "Yes" values
      
    )
  )

# Delete column: END_CUSTOMER
df = select (df,-c(END_CUSTOMER))

In [17]:
%%R

# Manipulate "ISIC" Feature: Convert to HAS_ISIC
df = df %>%
  mutate(
    HAS_ISIC = case_when(
      ISIC %in% c(NA) ~ 0,
      TRUE ~ 1 # Includes numbers
    )
  )

# Delete column: ISIC
df = select (df,-c(ISIC))

In [18]:
%%R

# Manipulate "COUNTRY_CODE" Feature: Convert to IS_COUNTRY_CODE_CH
df = df %>%
  mutate(
    IS_COUNTRY_CODE_CH = case_when(
      COUNTRY_CODE %in% c("CH") ~ 1,
      COUNTRY_CODE %in% c("FR") ~ 0,
    )
  )

# Delete column: COUNTRY_CODE
df = select (df,-c(COUNTRY_CODE))


In [19]:
%%R

# Manipulate "COSTS_PRODUCT_*" Features: Convert to HAS_PRODUCT_*
df = df %>%
  mutate(
    HAS_COSTS_PRODUCT_A = case_when(
      COSTS_PRODUCT_A %in% c(0) ~ 0,
      TRUE ~ 1 # Includes floating point numbers
    ),
    HAS_COSTS_PRODUCT_B = case_when(
      COSTS_PRODUCT_B %in% c(0) ~ 0,
      TRUE ~ 1 # Includes floating point numbers
    ),
    HAS_COSTS_PRODUCT_C = case_when(
      COSTS_PRODUCT_C %in% c(0) ~ 0,
      TRUE ~ 1 # Includes floating point numbers
    ),
    HAS_COSTS_PRODUCT_D = case_when(
      COSTS_PRODUCT_D %in% c(0) ~ 0,
      TRUE ~ 1 # Includes floating point numbers
    ),
    HAS_COSTS_PRODUCT_E = case_when(
      COSTS_PRODUCT_E %in% c(0) ~ 0,
      TRUE ~ 1 # Includes floating point numbers
    )
  )

### 2.3 Process Other Features

In [20]:
# Processed Features in 2.3:
#  * TOTAL_COSTS_PRODUCT      [Created]
#  * COSTS_PRODUCT_A          [Deleted]
#  * COSTS_PRODUCT_B          [Deleted]
#  * COSTS_PRODUCT_C          [Deleted]
#  * COSTS_PRODUCT_D          [Deleted]
#  * COSTS_PRODUCT_E          [Deleted]
#  * CREATION_YEAR            [Modified]
#  * SINCE_CREATION_YEAR      [Created]
#  * REV_CURRENT_YEAR.1       [Modified]
#  * REV_CURRENT_YEAR.2       [Modified]
#  * REV_PERCENTAGE_INCREASE  [Created]
#  * OWNERSHIP_NO_INFO_AS_NA  [Created]
#  * OWNERSHIP_NA_AS_NO_INFO  [Created]
#  * SO_CREATED_DATE_SCALED   [Created]
#  * SO_CREATED_DATE          [Deleted]
#  * SO_CREATED_DATE_INTEGER  [Deleted]


In [21]:
%%R

# Create "TOTAL_COSTS_PRODUCT" Feature: Sum of "COSTS_PRODUCT_*"
df = df %>%
  mutate(
    TOTAL_COSTS_PRODUCT = COSTS_PRODUCT_A
    + COSTS_PRODUCT_B + 
      COSTS_PRODUCT_C + COSTS_PRODUCT_D + COSTS_PRODUCT_E
  )

# Delete columns: COSTS_PRODUCT_A to COSTS_PRODUCT_E
df = select (df,-c(COSTS_PRODUCT_A,COSTS_PRODUCT_B,
                                 COSTS_PRODUCT_C,COSTS_PRODUCT_D,
                                 COSTS_PRODUCT_E))

In [22]:
%%R

# Manipulate "CREATION_YEAR" Feature: Extract year
df = df %>%
  mutate(CREATION_YEAR = case_when(
    is.character(CREATION_YEAR) ~ CREATION_YEAR %>%
      substr(nchar(CREATION_YEAR) - 3, nchar(CREATION_YEAR)) %>%
      as.numeric()
  ))

# Create "SINCE_CREATION_YEAR" Feature: 2021 - CREATION_YEAR
df = df %>%
  mutate(SINCE_CREATION_YEAR = case_when(!is.na(CREATION_YEAR) ~ as.double(2021 - CREATION_YEAR)))

In [23]:
%%R

# Manipulate "REV_CURRENT_YEAR.1" and "REV_CURRENT_YEAR.2" Feature: Convert to EUR

# Convert negative values to zero
# df$REV_CURRENT_YEAR.1 = ifelse(df$REV_CURRENT_YEAR.1 < 0,0,df$REV_CURRENT_YEAR.1)
# df$REV_CURRENT_YEAR.2 = ifelse(df$REV_CURRENT_YEAR.2 < 0,0,df$REV_CURRENT_YEAR.2)

# Create "REV_PERCENTAGE_INCREASE" Feature: REV_CURRENT_YEAR.2 to REV_CURRENT_YEAR.1
# calculate_percentage_increase = function(new, old) {
#   100 * (new - old) / old
# }
# df = df %>%
#   mutate(REV_PERCENTAGE_INCREASE = case_when(
#     !is.na(REV_CURRENT_YEAR.1) & !is.na(REV_CURRENT_YEAR.2) ~
#       calculate_percentage_increase(REV_CURRENT_YEAR.1, REV_CURRENT_YEAR.2)
#   ))


# df = df %>% mutate( # If REV_CURRENT_YEAR.1 is 0 or null, fill it from REV_CURRENT_YEAR.2
#   REV_CURRENT_YEAR.1 = ifelse(
#     ((REV_CURRENT_YEAR.1 <= 0) |
#        is.na(REV_CURRENT_YEAR.1)) &
#       !is.na(REV_CURRENT_YEAR.2),
#     REV_CURRENT_YEAR.2,
#     REV_CURRENT_YEAR.1
#   ) %>% as.double()
# )
# 
# df = df %>% mutate( # If REV_CURRENT_YEAR.2 is 0 or null, fill it from REV_CURRENT_YEAR.1
#   REV_CURRENT_YEAR.2 = ifelse(
#     ((REV_CURRENT_YEAR.2 <= 0) |
#        is.na(REV_CURRENT_YEAR.2)) &
#       !is.na(REV_CURRENT_YEAR.1),
#     REV_CURRENT_YEAR.1,
#     REV_CURRENT_YEAR.2
#   ) %>% as.double()
# )


# Note: 2020 and 2021 annual average exchange rates are used.
# Source: https://www.x-rates.com/average/?from=USD&to=EUR&amount=1&year=2021
df = df %>%
  mutate(
    REV_CURRENT_YEAR.1 = case_when(
      CURRENCY ==  "Pound Sterling" ~ REV_CURRENT_YEAR.1 * 1.1438161149110808,
      CURRENCY ==  "Chinese Yuan" ~ REV_CURRENT_YEAR.1 * 0.12906362243502054,
      CURRENCY ==  "US Dollar" ~ REV_CURRENT_YEAR.1 * 0.8614249616963066,
      CURRENCY ==  "Euro" ~ REV_CURRENT_YEAR.1
    )
  )


#df = df %>%
#  mutate(
#    REV_CURRENT_YEAR.2 = case_when(
#      CURRENCY ==  "Pound Sterling" ~ REV_CURRENT_YEAR.2 * 1.1438161149110808,
#      CURRENCY ==  "Chinese Yuan" ~ REV_CURRENT_YEAR.2 * 0.12906362243502054,
#      CURRENCY ==  "US Dollar" ~ REV_CURRENT_YEAR.2 * 0.8614249616963066,
#      CURRENCY ==  "Euro" ~ REV_CURRENT_YEAR.2
#    )
#  )

# Create "REV_RATE" Feature: REV_CURRENT_YEAR.2 to REV_CURRENT_YEAR.1
df = df %>%
  mutate(REV_RATE = case_when(
    !is.na(REV_CURRENT_YEAR.1) & !is.na(REV_CURRENT_YEAR.2)  &  REV_CURRENT_YEAR.1 !=0 & REV_CURRENT_YEAR.2 !=0 ~
     REV_CURRENT_YEAR.1/REV_CURRENT_YEAR.2
  ))

# Create "REV_AVG" Feature: REV_CURRENT_YEAR.2 + REV_CURRENT_YEAR.1
df = df %>%
  mutate(REV_AVG = case_when(
    !is.na(REV_CURRENT_YEAR.1) & !is.na(REV_CURRENT_YEAR.2)  &  REV_CURRENT_YEAR.1 !=0 & REV_CURRENT_YEAR.2 !=0 ~
     (REV_CURRENT_YEAR.1+REV_CURRENT_YEAR.2)/2,
     !is.na(REV_CURRENT_YEAR.1) & !is.na(REV_CURRENT_YEAR.2)  &  (REV_CURRENT_YEAR.1 ==0 | REV_CURRENT_YEAR.2 ==0) ~
     (REV_CURRENT_YEAR.1+REV_CURRENT_YEAR.2),
      !is.na(REV_CURRENT_YEAR.1) & !is.na(REV_CURRENT_YEAR.2)  &  REV_CURRENT_YEAR.1 ==0 & REV_CURRENT_YEAR.2 ==0 ~
      0.00
  ))

In [24]:
%%R

# Create "OWNERSHIP_NO_INFO_AS_NA" Feature: Treat "No information" as NA value
df$OWNERSHIP_NO_INFO_AS_NA = ifelse(df$OWNERSHIP ==  "No information", NA, df$OWNERSHIP)

# Create "OWNERSHIP_NA_AS_NO_INFO" Feature: Treat NA values as "No information"
df$OWNERSHIP_NA_AS_NO_INFO = ifelse(is.na(df$OWNERSHIP), "No information", df$OWNERSHIP)


# Create "SO_CREATED_DATE_SCALED" Feature: Scale "SO_CREATED_DATE" x 100

df$SO_CREATED_DATE_INTEGER = as_datetime(df$SO_CREATED_DATE, format = "%d.%m.%Y %H:%M") # Parse date-format 1
date_format_2 = as_datetime(df$SO_CREATED_DATE, format = "%Y-%m-%d %H:%M:%S") # Parse date-format 1
df$SO_CREATED_DATE_INTEGER[is.na(df$SO_CREATED_DATE_INTEGER)] = date_format_2[!is.na(date_format_2)]

df$SO_CREATED_DATE_INTEGER = as.numeric(as.POSIXct(df$SO_CREATED_DATE_INTEGER))# Convert to Unix Time Stamp
standart_scale = function (x) (x - mean(x, na.rm = T)) / sd(x, na.rm = T)

df$SO_CREATED_DATE_SCALED = standart_scale(df$SO_CREATED_DATE_INTEGER) # Scale x 100

# Delete column: "SO_CREATED_DATE"
df = select (df,-c(SO_CREATED_DATE))

In [25]:
%%R

# FIXME: Delete this before submission.
write_csv(x = df, file = "interim_data/df_completed_1_2.csv")

## 3. Dealing with Extreme Values, Filling Missing Values and Data Splitting

### 3.1 Deal with Extreme Values in Numeric Features

In [26]:
%%R

df$TOTAL_COSTS_PRODUCT = ifelse(df$TOTAL_COSTS_PRODUCT < 0,-df$TOTAL_COSTS_PRODUCT,df$TOTAL_COSTS_PRODUCT)
df$TOTAL_COSTS_PRODUCT_LOG=log(df$TOTAL_COSTS_PRODUCT+1)

df$SERVICE_COST = ifelse(df$SERVICE_COST < 0,-df$SERVICE_COST,df$SERVICE_COST)
df$SERVICE_COST_LOG=log(df$SERVICE_COST+1)

df$OFFER_PRICE_LOG=log(df$OFFER_PRICE)

df$SERVICE_LIST_PRICE_LOG=log(df$SERVICE_LIST_PRICE+1)

df$MATERIAL_COST_LOG=log(df$MATERIAL_COST+1)

df$REV_CURRENT_YEAR_LOG.1=log(df$REV_CURRENT_YEAR.1+1)

df$REV_CURRENT_YEAR_LOG.2=log(df$REV_CURRENT_YEAR.2+1)

df$CREATION_YEAR_LOG=log(df$CREATION_YEAR)

df$SINCE_CREATION_YEAR_LOG=log(df$SINCE_CREATION_YEAR+1)

#For REV_PERCENTAGE_INCREASE
#Q1 <- quantile(df$REV_PERCENTAGE_INCREASE, .25,na.rm=T)
#Q3 <- quantile(df$REV_PERCENTAGE_INCREASE, .75,na.rm=T)
#IQR <- IQR(df$REV_PERCENTAGE_INCREASE,na.rm=T)
#df = df %>%  mutate(
#  REV_PERCENTAGE_INCREASE_NO_OUTLIER = case_when(
#    REV_PERCENTAGE_INCREASE < (Q1 - 3.0*IQR) ~ (Q1 - 3.0*IQR),
#    REV_PERCENTAGE_INCREASE > (Q3 + 3.0*IQR) ~ (Q3 + 3.0*IQR),
#    TRUE ~ REV_PERCENTAGE_INCREASE
#  )
#)

### 3.2 Deal with Extreme Values in Character Features

In [27]:
%%R

df$TECH_REDUCED_1 = ifelse(df$TECH %in% c("E", "EPS", "FP", "BP"), "E_EPS_FP_BP", df$TECH)
df$TECH_REDUCED_2_IS_F = ifelse(df$TECH == "F", 1, 0)

df$OFFER_TYPE_REDUCED_1 = ifelse(
  df$OFFER_TYPE %in% c(
    "FD",
    "EH",
    "FEI",
    "MSYS",
    "DCF",
    "GAM",
    "CP",
    "CS",
    "CI",
    "EN",
    "FIB",
    "PAT",
    "XCPS"
  ),
  "FD_EH_FEI_MSYS_DCF_GAM_CP_CS_CI_EN_FIB_PAT_XCPS",
  df$OFFER_TYPE
)
df$OFFER_TYPE_REDUCED_2 = ifelse(
  df$OFFER_TYPE %in% c(
    "FED",
    "CPP",
    "ED",
    "EV",
    "FD",
    "EH",
    "FEI",
    "MSYS",
    "DCF",
    "GAM",
    "CP",
    "CS",
    "CI",
    "EN",
    "FIB",
    "PAT",
    "XCPS"
  ),
  "FED_CPP_ED_EV_FD_EH_FEI_MSYS_DCF_GAM_CP_CS_CI_EN_FIB_PAT_XCPS",
  df$OFFER_TYPE
)

df$OWNERSHIP_NA_AS_NO_INFO_REDUCED = ifelse(
  df$OWNERSHIP_NA_AS_NO_INFO %in% c("Governmental", "Individual Person", "No information"),
  "Governmental_IndividualPerson_Noinformation",
  df$OWNERSHIP_NA_AS_NO_INFO
)

df$OWNERSHIP_NO_INFO_AS_NA_REDUCED = ifelse(
  df$OWNERSHIP_NO_INFO_AS_NA %in% c("Governmental", "Individual Person"),
  "Governmental_IndividualPerson",
  df$OWNERSHIP_NO_INFO_AS_NA
)

df$OWNERSHIP_REDUCED = ifelse(
  df$OWNERSHIP %in% c("Governmental", "Individual Person", "No information"),
  "Governmental_IndividualPerson_Noinformation",
  df$OWNERSHIP
)

df$SALES_OFFICE_REDUCED = ifelse(
  df$SALES_OFFICE %in% c(
    "Montpellier",
    "Monaco",
    "Limoges",
    "Vertical Market",
    "Others Functions"
  ),
  "Montpellier_Monaco_Limoges_Vertical Market_OthersFunctions",
  df$SALES_OFFICE
)

### 3.3 Create IS_NA Columns and Remove the Duplicate Features

In [28]:
%%R

# For numeric columns
#df$IS_NA_REV_PERCENTAGE_INCREASE = ifelse(is.na(df$REV_PERCENTAGE_INCREASE) == T,1,0)
df$IS_NA_REV_RATE = ifelse(is.na(df$REV_RATE) == T,1,0)
df$IS_NA_REV_AVG = ifelse(is.na(df$REV_AVG) == T,1,0)
df$IS_NA_REV_CURRENT_YEAR = ifelse(is.na(df$REV_CURRENT_YEAR.1) == T,1,0)


# For Categoric Columns
df$IS_NA_SALES_LOCATION = ifelse(is.na(df$SALES_LOCATION) == T,1,0)
df$IS_NA_SALES_OFFICE = ifelse(is.na(df$SALES_OFFICE) == T,1,0)
df$IS_NA_CURRENCY = ifelse(is.na(df$CURRENCY) == T,1,0)
df$IS_NA_OWNERSHIP_NO_INFO_AS_NA = ifelse(is.na(df$OWNERSHIP_NO_INFO_AS_NA) == T,1,0)


In [29]:
%%R

# FIXME: Delete this before submission.
write_csv(x = df, file = "interim_data/df_completed_1_2_3_with_mv_new.csv")

### 3.4 Fill Missing Values in Numeric Features

In [ ]:
%%R

# Replace with mean

df$REV_CURRENT_YEAR.1[is.na(df$REV_CURRENT_YEAR.1)] = mean(df$REV_CURRENT_YEAR.1, na.rm=TRUE)

df$REV_CURRENT_YEAR.2[is.na(df$REV_CURRENT_YEAR.2)] = mean(df$REV_CURRENT_YEAR.2, na.rm=TRUE)

df$CREATION_YEAR[is.na(df$CREATION_YEAR)] = mean(df$CREATION_YEAR, na.rm=TRUE)

df$SINCE_CREATION_YEAR[is.na(df$SINCE_CREATION_YEAR)] = mean(df$SINCE_CREATION_YEAR, na.rm=TRUE)

df$REV_CURRENT_YEAR_LOG.1[is.na(df$REV_CURRENT_YEAR_LOG.1)] = mean(df$REV_CURRENT_YEAR_LOG.1, na.rm=TRUE)

df$REV_CURRENT_YEAR_LOG.2[is.na(df$REV_CURRENT_YEAR_LOG.2)] = mean(df$REV_CURRENT_YEAR_LOG.2, na.rm=TRUE)

df$CREATION_YEAR_LOG[is.na(df$CREATION_YEAR_LOG)] = mean(df$CREATION_YEAR_LOG, na.rm=TRUE)

df$SINCE_CREATION_YEAR_LOG[is.na(df$SINCE_CREATION_YEAR_LOG)] = mean(df$SINCE_CREATION_YEAR_LOG, na.rm=TRUE)

df$REV_PERCENTAGE_INCREASE_NO_OUTLIER[is.na(df$REV_PERCENTAGE_INCREASE_NO_OUTLIER)] = mean(df$REV_PERCENTAGE_INCREASE_NO_OUTLIER, na.rm=TRUE)

df$REV_PERCENTAGE_INCREASE[is.na(df$REV_PERCENTAGE_INCREASE)] = mean(df$REV_PERCENTAGE_INCREASE, na.rm=TRUE)


### 3.5 Fill Missing Values in Character Features


In [ ]:
%%R

df$SALES_LOCATION[is.na(df$SALES_LOCATION)] = "Geneva West"

df$SALES_OFFICE[is.na(df$SALES_OFFICE)] = "Geneva"

df$SALES_OFFICE_REDUCED[is.na(df$SALES_OFFICE_REDUCED)] = "Geneva"

df$CURRENCY[is.na(df$CURRENCY)] = "NOT_GIVEN"

df$OWNERSHIP[is.na(df$OWNERSHIP)] = "NOT_GIVEN"

df$OWNERSHIP_NO_INFO_AS_NA[is.na(df$OWNERSHIP_NO_INFO_AS_NA)] = "NOT_GIVEN"

df$OWNERSHIP_NO_INFO_AS_NA_REDUCED[is.na(df$OWNERSHIP_NO_INFO_AS_NA_REDUCED)] = "NOT_GIVEN"

df$OWNERSHIP_REDUCED[is.na(df$OWNERSHIP_REDUCED)] = "NOT_GIVEN"


In [ ]:
%%R

# FIXME: Delete this before submission.
write_csv(x = df, file = "interim_data/df_completed_1_2_3.csv")

## 4. Feature Importance

In [ ]:
%%R

install.packages("party")
library(party)

In [ ]:
%%R

x = df

x = x[is.na(x$OFFER_STATUS) == F,]

x = x %>% select(-c(TEST_SET_ID,IDX_CUSTOMER,CUSTOMER))

In [ ]:
%%R

x$OFFER_STATUS = as.numeric(x$OFFER_STATUS)

a <- select_if(x, is.numeric)  
colnames(a)
cf1 <- cforest(a$OFFER_STATUS ~ . , data= a, control=cforest_unbiased(mtry=2,ntree=50)) # fit the random forest


In [ ]:
%%R

varimp(cf1) # get variable importance, based on mean decrease in accuracy

In [ ]:
%%R

#install.packages("earth")
library(earth)

marsModel <- earth(x$OFFER_STATUS ~ ., data=x) # build model
ev <- evimp (marsModel) # estimate variable importance

plot(ev)

In [ ]:
%%R

#install.packages("relaimpo")
library(relaimpo)
lmMod <- lm(OFFER_STATUS ~ . , data = x)  # fit lm() model
relImportance <- calc.relimp(lmMod, type = "lmg", rela = TRUE)  # calculate relative importance scaled to 100
sort(relImportance$lmg, decreasing=TRUE)  # relative importance

In [ ]:

names(boruta_output)

# Get significant variables including tentatives
boruta_signif <-
  getSelectedAttributes(boruta_output, withTentative = TRUE)
print(boruta_signif)

# Do a tentative rough fix
roughFixMod <- TentativeRoughFix(boruta_output)
boruta_signif <- getSelectedAttributes(roughFixMod)
print(boruta_signif)

# Variable Importance Scores
imps <- attStats(roughFixMod)
imps2 = imps[imps$decision != 'Rejected', c('meanImp', 'decision')]
head(imps2[order(-imps2$meanImp),])  # descending sort

x = summary(boruta_output$ImpHistory)
# Plot variable importance
plot(
  boruta_output,
  cex.axis = .7,
  las = 2,
  xlab = "",
  main = "Variable Importance"
)

In [ ]:
%%R

print("[INFO] Number of NA values in each column:")
for (i in 1:ncol(df)) {
  print(paste0(names(df)[i], ": ", sum(is.na(df[, i])), "/", nrow(df)))
}

In [ ]:
%%R

# fill in missing values
customers = customers %>% mutate(REV_CURRENT_YEAR.1 = ifelse(REV_CURRENT_YEAR.1 == 0,REV_CURRENT_YEAR.2, REV_CURRENT_YEAR.1))
customers = customers %>% mutate(REV_CURRENT_YEAR = ifelse(REV_CURRENT_YEAR == 0,REV_CURRENT_YEAR.2, REV_CURRENT_YEAR))

## Li - Section

In [ ]:
%%R
## check NA values in the labeled_data
#print("[INFO] Number of NA values in each column:")
#for (i in 1:ncol(labeled_data)) { # for-loop over columns
#  print(paste0(names(labeled_data)[i], ": ", sum(is.na(labeled_data[, i])), "/", nrow(labeled_data)))
#}

In [ ]:
%%R
#remove REV_CURRENT_YEAR column as it's the same as REV_CURRENT_YEAR.1
all_merged = subset(all_merged, select = -c(REV_CURRENT_YEAR))
#remove MO_CREATED_DATE column
all_merged = subset(all_merged, select = -c(MO_CREATED_DATE))

In [ ]:
%%R

#nice function from stackoverflow 
one_hot_encoding = function(df, columns="PRICE_LIST"){
  # create a copy of the original data.frame for not modifying the original
  df = cbind(df)
  # convert the columns to vector in case it is a string
  columns = c(columns)
  # for each variable perform the One hot encoding
  for (column in columns){
    unique_values = sort(unique(df[column])[,column])
    non_reference_values  = unique_values[c(-1)] # the first element is going 
                                                 # to be the reference by default
    for (value in non_reference_values){
      # the new dummy column name
      new_col_name = paste0(column,'_',value)
      # create new dummy column for each value of the non_reference_values
      df[new_col_name] <- with(df, ifelse(df[,column] == value, 1, 0))
    }
    # delete the one hot encoded column
    df[column] = NULL

  }
  return(df)
}

In [ ]:
%%R
#one_hot_encoding column PRICE_LIST, TECH, BUSINESS_TYPE, OFFER_TYPE
all_merged = one_hot_encoding(all_merged, c("PRICE_LIST"))
all_merged = one_hot_encoding(all_merged, c("TECH"))
all_merged = one_hot_encoding(all_merged, c("BUSINESS_TYPE"))
all_merged = one_hot_encoding(all_merged, c("OFFER_TYPE"))

In [ ]:
%%R
#Convert COUNTRY_CODE to binary
all_merged = all_merged %>%
  mutate(
    COUNTRY_CODE = case_when(
      COUNTRY_CODE %in% c("CH") ~ 0,
      COUNTRY_CODE %in% c("FR") ~ 1,
    ))

In [ ]:
%%R
glimpse(all_merged)
write_csv(x = all_merged, "interim_data/all_merged_3.csv")

In [ ]:
%%R

#install.packages("Hmisc")
#library(Hmisc)
#complete REV_CURRENT_YEAR.1 and REV_CURRENT_YEAR.2 by mean using impute method
#labeled_data$REV_CURRENT_YEAR.1 = impute(labeled_data$REV_CURRENT_YEAR.1, mean)
#labeled_data$REV_CURRENT_YEAR.2 = impute(labeled_data$REV_CURRENT_YEAR.2, mean)

In [ ]:
%%R

#print("[INFO] Number of NA values in each column:")
#for (i in 1:ncol(labeled_data)) { # for-loop over columns
#  print(paste0(names(labeled_data)[i], ": ", sum(is.na(labeled_data[, i])), "/", nrow(labeled_data)))
#}

In [ ]:
drive.flush_and_unmount()

## Saqib - Section

In [ ]:
%%R

all_merged = one_hot_encoding(all_merged, c("SALES_OFFICE"))
all_merged = one_hot_encoding(all_merged, c("SALES_BRANCH"))

glimpse(all_merged)

## Teofil - Section

In [ ]:
%%R

old_all_merged = all_merged

In [ ]:
%%R

customers = cst_df


cst_df = cst_df %>% mutate(REV_CURRENT_YEAR.1 = ifelse(REV_CURRENT_YEAR.1 == 0,REV_CURRENT_YEAR.2, REV_CURRENT_YEAR.1))
cst_df = cst_df %>% mutate(PREV_YEAR_PERCENTAGE_INCREASE.1 = ((REV_CURRENT_YEAR.1 - REV_CURRENT_YEAR.2)/REV_CURRENT_YEAR.2)*100)


################# REPLACEMENT BLOC FOR FINAL MERGE ########################
cst_df = cst_df %>% mutate(COUNTRY_CODE = case_when(
  COUNTRY == 'Switzerland' ~ "CH",
  COUNTRY == 'France' ~ "FR"
))
# Transform customer to integer
trnsc_geo_df$CUSTOMER <- as.numeric(trnsc_geo_df$CUSTOMER)
all_merged = left_join(trnsc_geo_df, cst_df, by = c("CUSTOMER", "COUNTRY_CODE"))
#################################################################################


########## SPLIT LABELED DATA INTO TRAIN AND VALIDATION BASED ON UNIQUE CUSTOMERS ##############
unique_customers = unique(labeled_data$CUSTOMER)
train_ids = sample(unique_customers, size= floor(0.8 * length(unique_customers)), replace=FALSE)

train_set = labeled_data %>% filter(CUSTOMER %in% train_ids)
validation_set = labeled_data %>%  filter(!CUSTOMER %in% train_ids)

In [1]:
!pip install missingno > /dev/null
!pip install category_encoders > /dev/null

#Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import missingno as msno
from sklearn.model_selection import train_test_split
from sklearn import base
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor, MLPClassifier
from category_encoders import BinaryEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,roc_auc_score,classification_report,confusion_matrix
from IPython.display import Image
import warnings
import pandas as pd
import numpy as np
import collections as c
import sklearn
import os

from sklearn.preprocessing import MultiLabelBinarizer
import category_encoders as ce
from sklearn import preprocessing

# use feature importance for feature selection
from numpy import loadtxt
from numpy import sort
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.feature_selection import SelectFromModel

# example of auto-sklearn for a classification dataset
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from autosklearn.classification import AutoSklearnClassifier
import autosklearn

import copy
import time
import pickle
import itertools
from typing import List
import datetime

%load_ext nb_black

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 1000)  # or None
pd.set_option("display.max_rows", 1000)  # or None
pd.set_option("display.max_colwidth", -1)  # or -1



/home/iceking/Desktop/TUM Lecture Docs/3. Business Analytics and Machine Learning (IN2028)/Analytics Cup/Project Files/venv/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex
/home/iceking/Desktop/TUM Lecture Docs/3. Business Analytics and Machine Learning (IN2028)/Analytics Cup/Project Files/venv/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


<IPython.core.display.Javascript object>

In [61]:

## Helper Functions

def type_separator(df: pd.DataFrame, print_results=False):
    dtype_names = ["categorical", "binary", "continuous", "integer", "numerical"]
    type_to_cols = dict.fromkeys(dtype_names, [])

    dtype_char_to_names = {
        "O": "categorical",
        "i": "integer",
        "u": "integer",
        "f": "continuous",
    }

    for col in df.columns:
        unique_vals = set(df[col].dropna().unique())

        if len(unique_vals) < 2 or (
            df[col].dtype == np.dtype("object") and len(unique_vals) > 500
        ):
            raise ValueError(
                f"[ERROR] Something wrong with column:{col} cannot be this case, check conversions!"
            )

        if unique_vals == {0, 1} or unique_vals == {0.0, 1.0}:
            type_to_cols["binary"] = type_to_cols["binary"] + [col]
        elif df[col].dtype.str[1] in dtype_char_to_names:
            dtype_char = df[col].dtype.str[1]
            dtype_name = dtype_char_to_names[dtype_char]
            type_to_cols[dtype_name] = type_to_cols[dtype_name] + [col]
        else:
            raise ValueError(
                f'[ERROR] Numpy data type:"{df[col].dtype}" of col:"{col}" not understood.'
            )

    type_to_cols["numerical"] = type_to_cols["continuous"] + type_to_cols["integer"]
    type_to_cols["nominal"] = type_to_cols["categorical"] + type_to_cols["binary"]
    for dtype_name, col_names in type_to_cols.items():
        assert len(type_to_cols[dtype_name]) == len(
            set(type_to_cols[dtype_name])
        ), f'For type:"{dtype_name}", some columns are duplicate in: {col_names}.'

    if print_results:
        for key, val in type_to_cols.items():
            print("type:", key, "columns:")
            for col in sorted(val):
                print(">", col)
            print("-" * 32)
    return type_to_cols


def get_null_columns(df: pd.DataFrame) -> List:
    return [col for col in df.columns if np.any(df[col].isna())]


def get_non_null_columns(df: pd.DataFrame) -> List:
    return [col for col in df.columns if not np.any(df[col].isna())]


def get_inf_columns(df: pd.DataFrame) -> List:
    return [col for col in df.columns if np.any(df[col] == np.inf)]


def print_nan_and_inf_columns(df: pd.DataFrame):
    print("NaN and Infinity Columns and Counts:")
    for col in sorted(get_null_columns(df)):
        print("> [NaN     ]", col, "Null Count:", np.sum(df[col].isna()))
    for col in sorted(get_inf_columns(df)):
        print("> [INFINITY]", col, "Inf Count:", np.sum(df[col] == np.inf))


def get_labeled_set(df: pd.DataFrame, target_col: str = "OFFER_STATUS"):
    return df[~np.isnan(df[target_col])]


def get_unlabeled_set(df: pd.DataFrame, target_col: str = "OFFER_STATUS"):
    return df[np.isnan(df[target_col])]


def MissingUniqueStatistics(df, show_unique_values=False):

    total_entry_list = []
    total_missing_value_list = []
    missing_value_ratio_list = []
    data_type_list = []
    unique_values_list = []
    number_of_unique_values_list = []
    variable_name_list = []

    for col in df.columns:

        variable_name_list.append(col)
        missing_value_ratio = round((df[col].isna().sum() / len(df[col])), 4)
        total_entry_list.append(df[col].shape[0] - df[col].isna().sum())
        total_missing_value_list.append(df[col].isna().sum())
        missing_value_ratio_list.append(missing_value_ratio)
        data_type_list.append(df[col].dtype)
        unique_values_list.append(list(df[col].unique()))
        number_of_unique_values_list.append(len(df[col].unique()))

    data_info_df = pd.DataFrame(
        {
            "Variable": variable_name_list,
            "#_Total_Entry": total_entry_list,
            "#_Missing_Value": total_missing_value_list,
            "%_Missing_Value": missing_value_ratio_list,
            "Data_Type": data_type_list,
            "Unique_Values": unique_values_list,
            "#_Uniques_Values": number_of_unique_values_list,
        }
    )
    if not show_unique_values:
        data_info_df = data_info_df.drop("Unique_Values", axis=1)

    return data_info_df.sort_values(by="#_Missing_Value", ascending=False).set_index(
        "Variable"
    )


def histogram(df, feature):  # Histogram of the target categories
    %matplotlib inline
    ncount = len(df)
    ax = sns.countplot(x=feature, data=df, palette="hls")
    sns.set(font_scale=1)
    ax.set_xlabel("Target Segments")
    plt.xticks(rotation=90)
    ax.set_ylabel("Number of Observations")
    fig = plt.gcf()
    fig.set_size_inches(12, 5)
    # Make twin axis
    ax2 = ax.twinx()
    # Switch so count axis is on right, frequency on left
    ax2.yaxis.tick_left()
    ax.yaxis.tick_right()
    # Also switch the labels over
    ax.yaxis.set_label_position("right")
    ax2.yaxis.set_label_position("left")
    ax2.set_ylabel("Frequency [%]")
    for p in ax.patches:
        x = p.get_bbox().get_points()[:, 0]
        y = p.get_bbox().get_points()[1, 1]
        ax.annotate(
            "{:.2f}%".format(100.0 * y / ncount),
            (x.mean(), y),
            ha="center",
            va="bottom",
        )  # set the alignment of the text
    # Use a LinearLocator to ensure the correct number of ticks
    ax.yaxis.set_major_locator(ticker.LinearLocator(11))
    # Fix the frequency range to 0-100
    ax2.set_ylim(0, 100)
    ax.set_ylim(0, ncount)
    # And use a MultipleLocator to ensure a tick spacing of 10
    ax2.yaxis.set_major_locator(ticker.MultipleLocator(10))
    # Need to turn the grid on ax2 off, otherwise the gridlines end up on top of the bars
    ax2.grid(None)
    plt.title("Histogram of Binary Target Categories", fontsize=20, y=1.08)
    plt.show()
    plt.savefig("target_histogram.png")
    del ncount, x, y

    # USAGE: histogram(data, "CLASS")

## From Submission 3, With H2O

def apply_h2o(data, max_runtime_secs=10 * 60, max_models=20, balance_classes=True,
              models_dir_path = './h2o_models_with_data',training_with_all=False):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
    X_train, X_test, y_train, y_test = data
    
    from h2o.automl import H2OAutoML
    import h2o
    h2o.init(max_mem_size="32G")
    
    if not os.path.exists(models_dir_path):
        os.makedirs(models_dir_path)
        
    results_dir_path =f'{models_dir_path}/results_ts_{timestamp}'
    os.makedirs(results_dir_path)
        
    train_path =f"{results_dir_path}/train.csv"
    test_path =f"{results_dir_path}/test.csv"

    pd.concat([X_train, y_train], axis=1).to_csv(train_path, index=False, header=True)
    pd.concat([X_test, y_test], axis=1).to_csv(test_path, index=False, header=True)

    train = h2o.import_file(train_path)
    test = h2o.import_file(test_path)

    h2o.init(max_mem_size="36G")

    x = train.columns
    train["OFFER_STATUS"] = train["OFFER_STATUS"].asfactor()
    x.remove("OFFER_STATUS")
    aml = H2OAutoML(
        max_models=max_models,
        balance_classes=balance_classes,
        max_runtime_secs=int(max_runtime_secs),
        seed=42,
    )
    if  training_with_all:
        saved_model_path=f'{results_dir_path}/model_withALL'
        print(f'[INFO] Model will be saved here:"{saved_model_path}"')
    
    
    aml.train(x=x, y="OFFER_STATUS", training_frame=train)
    
    print("[INFO] Timestamp:",timestamp)
    print('[INFO] AML Leaderboard',aml.leaderboard)
    
    model_bac_scores = []
    
    if  training_with_all:
        saved_model_path=f'{results_dir_path}/model_withALL'
        h2o.save_model(model=aml.leader, path=saved_model_path, force=True)
    else:
        for i in range(int(aml.max_models * 1.5)):
            current_model = aml.leaderboard[i, 0]
            if current_model == "NA":
                print(f"[INFO] Found {i} models in total.")
                break
            current_model = h2o.get_model(current_model)
            new_pred = current_model.predict(test)
            new_pred = new_pred[0].as_data_frame().values.flatten()
            model_bac_score = balanced_accuracy_score(y_test, new_pred)
            print(f'[INFO] Model #{i}, BAC={model_bac_score}.')
            model_bac_scores.append(model_bac_score)

        index_max_bac_score = model_bac_scores.index(max(model_bac_scores))
    
        saved_model_path=f'{results_dir_path}/model_{"bac_%.3f" % model_bac_scores[index_max_bac_score]}'
        h2o.save_model(model=h2o.get_model(aml.leaderboard[index_max_bac_score, 0]), 
                   path=saved_model_path, force=True)
    
    print("[INFO] RESULTS:")
    print(f' > Train data saved to:"{train_path}".')
    print(f' > Test data saved to :"{test_path}".')
    print(f' > H20 Model saved to :"{saved_model_path}".')

    return aml, model_bac_scores


<IPython.core.display.Javascript object>

In [62]:
def code_block(params:dict, verbose=False, calc_feature_importances=False):
    
    ####### Get Parameters - Start ################
    df = pd.read_csv(params['preprocessed_data_path']) # Read Data
    ###############################################
    
    ####### Block Specific Helper Functions #######
    def print_if_verbose(*args, **kwargs):
        if verbose:
            print(*args, **kwargs)
    ###############################################

    ###############################################
    id_columns = ["CUSTOMER", "TEST_SET_ID", "IDX_CUSTOMER"]
    #unnecessary_reduced_cols = [
    #    "OFFER_TYPE_REDUCED_1",
    #    "OFFER_TYPE_REDUCED_2",
    #    "SALES_OFFICE_REDUCED",
    #]

    #to_be_dropped_cols = id_columns + unnecessary_reduced_cols
    to_be_dropped_cols = id_columns
    df = df.drop(to_be_dropped_cols, axis=1)

    # new columns
    df["ADDITIONAL_COST"] = df["OFFER_PRICE"] - df["MATERIAL_COST"] - df["SERVICE_COST"]
    df["TOTAL_COST"] = df["MATERIAL_COST"] + df["SERVICE_COST"]

    ###############################################

    assert 'OFFER_STATUS' not in type_separator(df)["categorical"]
    for col in type_separator(df)["categorical"]:
        num_unq = len(df[col].unique())
        trimmed_col = col.strip().replace(" ", "_")
        if num_unq < 5:
            print_if_verbose(f"[INFO] Col:{col},num_of_unq:{num_unq}, applying 1-HOT encoding.")
            onehot_df = pd.get_dummies(df[col])
            onehot_df = onehot_df.add_prefix(trimmed_col + "_1HOTENC_")
            df = pd.concat((df, onehot_df), axis=1)
        elif num_unq >= 5:
            print_if_verbose(f"[INFO] Col:{col},num_of_unq:{num_unq}, applying BINARY encoding.")
            encoder = ce.BinaryEncoder(cols=[col])
            binenc_df = encoder.fit_transform(df[[col]])
            binenc_df.columns = [
                f"{trimmed_col}_BINENC_{i}" for i in range(len(binenc_df.columns))
            ]
            df = pd.concat((df, binenc_df), axis=1)

    for col in type_separator(df)["categorical"]:
        if col != 'OFFER_STATUS':
            df[col] = pd.Categorical(df[col])

    ##########################

    ###### ADDITIONAL < FEATURES - START
    add_less_than_features = True
    print("[WARN] Adding Less Than Features:", add_less_than_features)

    if add_less_than_features:
        raw_num_cols = type_separator(df)["numerical"]
        nonraw_strings_in_cols = ("OFFER_STATUS","IS_NA","HAS_","1HOTENC","BINENC","IS_","_LOG")
        raw_num_cols = [a for a in raw_num_cols if not any( x in a for x in nonraw_strings_in_cols)]

        raw_numeric_cols_combinations = list(itertools.combinations(raw_num_cols, r=2))
        new_less_than_cols = []
        for col1, col2 in raw_numeric_cols_combinations:
            new_col = np.where(
                np.isnan(df[col1]) | np.isnan(df[col2]), np.nan, (df[col1] < df[col2])
            )
            name_of_new_col = col1 + "_<_" + col2

            if (len(np.unique(new_col[~np.isnan(new_col)])) > 1):  # Not all values are 1 or 0
                new_less_than_cols.append(pd.Series(new_col, name=name_of_new_col))
                print_if_verbose(f'[INFO] Added new "less than" column: "{name_of_new_col}".')
            else:
                print_if_verbose(f'[INFO] NOT added "Less Then" column: "{name_of_new_col}".')

        df = pd.concat([df, pd.concat(new_less_than_cols, axis=1)], axis=1)

    ############# DROPPP -START
    drop_cols = [
    #    ############## Correlation are same or very similar
    #    "OWNERSHIP_NA_AS_NO_INFO_1HOTENC_Privately Owned/Publicly Traded",
    #    "OWNERSHIP_NA_AS_NO_INFO_REDUCED_1HOTENC_Privately Owned/Publicly Traded",
    #    "OWNERSHIP_NO_INFO_AS_NA_1HOTENC_Privately Owned/Publicly Traded",
    #    "OWNERSHIP_NO_INFO_AS_NA_REDUCED_1HOTENC_Privately Owned/Publicly Traded",
    #    "OWNERSHIP_NA_AS_NO_INFO_1HOTENC_Individual Person",
    #    "OWNERSHIP_NA_AS_NO_INFO_1HOTENC_Governmental",
    #    "OWNERSHIP_NA_AS_NO_INFO_1HOTENC_No information",
    #    "OWNERSHIP_NO_INFO_AS_NA_REDUCED_1HOTENC_NOT_GIVEN",
    #    "OWNERSHIP_NO_INFO_AS_NA_1HOTENC_NOT_GIVEN",
    #    "OWNERSHIP_REDUCED_1HOTENC_NOT_GIVEN",
    #    "TECH_REDUCED_2_IS_F",
    #    "TECH_BINENC_0",
    #    "SINCE_CREATION_YEAR_<_REV_PERCENTAGE_INCREASE_NO_OUTLIER",  # (34, False)
    #    ############## Experimentally
    #    "TOTAL_COSTS_PRODUCT_LOG",
    #    "CURRENCY_BINENC_0",  # (19, False)
    #    "OWNERSHIP_BINENC_0",  # (20, False)
    #    "OWNERSHIP_NO_INFO_AS_NA_1HOTENC_Individual Person",  # (21, False)
    #    "IS_NA_SALES_LOCATION",  # (22, False)
    #    "IS_NA_SALES_OFFICE",  # (23, False)
    #    ## From XGB Feature Importance
    #    "SERVICE_COST_<_CREATION_YEAR",  # 0.0
    #    "REV_CURRENT_YEAR.1_<_REV_PERCENTAGE_INCREASE",  # 0.0
    #    "CREATION_YEAR_<_TOTAL_COST",  # 0.0
    #    "CREATION_YEAR_<_REV_PERCENTAGE_INCREASE",  # 0.0
    #    "CREATION_YEAR_<_ADDITIONAL_COST",  # 0.0
    ]

    for col in drop_cols:
        if col in list(df.columns):
            print_if_verbose("[INFO] Dropped:", col)
            df = df.drop(col, axis=1)

    if calc_feature_importances:
        def calculate_feature_importances():
            from boruta import BorutaPy
            from sklearn.ensemble import RandomForestClassifier

            df_new = df[~np.isnan(df["OFFER_STATUS"])].copy()
            columns_to_be_dropped = get_null_columns(df_new)
            if "categorical" in type_separator(df_new):
                columns_to_be_dropped += [
                    col for col in type_separator(df_new)["categorical"]
                    if col in df_new.columns
                ]
            columns_to_be_dropped = set(columns_to_be_dropped)
            if "OFFER_STATUS" in columns_to_be_dropped:
                columns_to_be_dropped.remove("OFFER_STATUS")

            df_new = df_new.drop(columns_to_be_dropped, axis=1)
            X, y = df_new.drop("OFFER_STATUS", axis=1), df_new["OFFER_STATUS"]

            forest = RandomForestClassifier(n_jobs=-1, class_weight="balanced", max_depth=5)
            forest.fit(X, y)
            
            feat_selector = BorutaPy( # define Boruta feature selection method
                forest, n_estimators="auto", verbose=2,
                random_state=42,
            )
            
            feat_selector.fit(X.to_numpy(), y.to_numpy()) # find all relevant features
            feature_importances = list(zip(feat_selector.ranking_,feat_selector.support_,X.columns))
            for item in sorted(feature_importances):
                print(item, ",")
        calculate_feature_importances()
        
    if params['train_all_mode']:
        df_for_unlabeled_set = df[np.isnan(df["OFFER_STATUS"])]
        df_for_labeled_set = df[~np.isnan(df["OFFER_STATUS"])]
        
        X_train, y_train = df_for_labeled_set.drop(["OFFER_STATUS"], axis=1), df_for_labeled_set["OFFER_STATUS"]
        X_test = df_for_unlabeled_set.drop(["OFFER_STATUS"], axis=1)
        y_test = df_for_unlabeled_set["OFFER_STATUS"]
    else:
        df_for_test = df[np.isnan(df["OFFER_STATUS"])]
        df = df[~np.isnan(df["OFFER_STATUS"])]

        X, y = df.drop("OFFER_STATUS", axis=1), df["OFFER_STATUS"] # Col Selection & Conversion
        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.33, random_state=42)
    
    if params['classifer'] == 'h2o':
        return apply_h2o(data=[X_train, X_test, y_train, y_test],max_runtime_secs=params['max_runtime_secs'],
                         training_with_all=params['train_all_mode'],max_models=params["max_models"])


<IPython.core.display.Javascript object>

In [63]:
result = code_block(
    params={
        "max_runtime_secs": 5 * 60 * 60,
        # "max_runtime_secs": 30,
        "preprocessed_data_path": "interim_data/df_completed_1_2_3_with_mv_new.csv",
        "classifer": "h2o",
        "train_all_mode": False,
        "max_models": 400,
    }
)
result

[WARN] Adding Less Than Features: True
Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 hours 58 mins
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.2
H2O_cluster_version_age:,4 days
H2O_cluster_name:,H2O_from_python_iceking_xb133p
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,30.82 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 hours 58 mins
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.2
H2O_cluster_version_age:,4 days
H2O_cluster_name:,H2O_from_python_iceking_xb133p
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,30.82 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


AutoML progress: |██
20:22:31.349: _train param, Dropping bad and constant columns: [REV_RATE_<_SO_CREATED_DATE_INTEGER]

████████████████████████████████████████
20:34:03.867: XRT_1_AutoML_25_20220129_202209 [DRF XRT (Extremely Randomized Trees)] failed: java.lang.AssertionError

█
20:34:40.879: _train param, Dropping bad and constant columns: [REV_RATE_<_SO_CREATED_DATE_INTEGER]

████████████████████
01:13:12.370: GBM_lr_annealing_selection_AutoML_25_20220129_202209 [GBM lr_annealing] failed: water.exceptions.H2OIllegalArgumentException: Can only convert jobs producing a single Model or ModelContainer.

| (done) 100%
[INFO] Timestamp: 2022-01-29_20:22:06
[INFO] AML Leaderboard 

...


stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
[INFO] Model #0, BAC=0.6561937173174357.
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
[INFO] Model #1, BAC=0.6424053322078234.
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
[INFO] Model #2, BAC=0.6527547869448792.
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
[INFO] Model #3, BAC=0.6417652297914368.
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
[INFO] Model #4, BAC=0.6526813068667081.
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
[INFO] Model #5, BAC=0.6642162680712127.
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
[INFO] Model #6, BAC=0.6395656877928686.
stackedensemble prediction progress: |█████████

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
[INFO] Model #60, BAC=0.6601698989016558.
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
[INFO] Model #61, BAC=0.641206812554411.
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
[INFO] Model #62, BAC=0.6294448783915159.
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
[INFO] Model #63, BAC=0.6887215605705274.
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
[INFO] Model #64, BAC=0.7017131101524905.
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
[INFO] Model #65, BAC=0.6825964786632354.
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
[INFO] Model #66, BAC=0.6262133228057182.
xgboost prediction progress: |████████████

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
[INFO] Model #120, BAC=0.6845822176747496.
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
[INFO] Model #121, BAC=0.675103026281723.
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
[INFO] Model #122, BAC=0.6765971386251324.
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
[INFO] Model #123, BAC=0.6968846955421424.
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
[INFO] Model #124, BAC=0.6901489346068068.
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
[INFO] Model #125, BAC=0.672157290431002.
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
[INFO] Model #126, BAC=0.6920544517691087.
deeplearning prediction progress: |█

(<h2o.automl._estimator.H2OAutoML at 0x7fa0de1a5af0>,
 [0.6561937173174357,
  0.6424053322078234,
  0.6527547869448792,
  0.6417652297914368,
  0.6526813068667081,
  0.6642162680712127,
  0.6395656877928686,
  0.6801911297316389,
  0.6744661118345885,
  0.6297224407636116,
  0.6349935127438809,
  0.6396310150316409,
  0.660770804974779,
  0.7215253983419843,
  0.7313295012897687,
  0.7157970879520997,
  0.7268553170989396,
  0.728832746485682,
  0.7222896225120383,
  0.7223761680379638,
  0.6974789121060154,
  0.7206273317251335,
  0.688556622358075,
  0.6814615616094917,
  0.7229493231000566,
  0.7163016755443772,
  0.7231142613125089,
  0.7252925327621329,
  0.6900425818620856,
  0.7196018508617813,
  0.7217605241397735,
  0.6930667626611244,
  0.6214647642120453,
  0.725790639892324,
  0.7160338338654106,
  0.681113759390268,
  0.7145054900488846,
  0.6843208041960782,
  0.707485477232203,
  0.6762607294763505,
  0.6781467529906026,
  0.6909080371213411,
  0.6445738829643181,
  0.71

<IPython.core.display.Javascript object>

In [ ]:
%de

In [1]:
import pickle

In [ ]:
pickle.dump(result)

In [136]:
def experimental(
    model_path: str,
    test_df_or_path=None,
    with_real_test: bool = False,
    output_name=None,
):
    from h2o.automl import H2OAutoML
    import h2o

    h2o.init(max_mem_size="32G")

    if with_real_test:
        test_df = pd.read_csv("interim_data/df_completed_1_2_3_with_mv_new.csv")
        test = h2o.H2OFrame(test_df[np.isnan(test_df["OFFER_STATUS"])])
    else:
        if type(test_df_or_path) == str:  # Path
            test = h2o.import_file(test_df_or_path)
        else:
            test = h2o.H2OFrame(test_df_or_path)

    my_model = h2o.load_model(model_path)

    test["OFFER_STATUS"] = test["OFFER_STATUS"].asfactor()
    x_test = test.drop("OFFER_STATUS", axis=1)

    pred = my_model.predict(x_test)
    return pred
    pred = pred[0].as_data_frame().values.flatten()
    if with_real_test:
        final_pred_df = pd.DataFrame()
        final_pred_df["prediction"] = pred
        final_pred_df["id"] = test["TEST_SET_ID"]

        final_pred_df["prediction"] = final_pred_df["prediction"].astype(int)
        final_pred_df.to_csv(output_name, header=True, index=False)
        return final_pred_df

    y_test = test["OFFER_STATUS"]
    y_test = y_test.as_data_frame().values.flatten()
    model_bac_score = balanced_accuracy_score(y_test, pred)
    print(f"[INFO] Model, BAC={model_bac_score}.")


experimental_result = experimental(
    model_path="./h2o_models_with_data/results_ts_2022-01-29_20:22:06/model_bac_0.731/GBM_grid_1_AutoML_25_20220129_202209_model_16",
    test_df_or_path="./h2o_models_with_data/results_ts_2022-01-29_20:22:06/test.csv",
    with_real_test=True,
    output_name="predictions_versed_chimpanzee_8.csv",
)



Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,9 hours 32 mins
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.2
H2O_cluster_version_age:,4 days
H2O_cluster_name:,H2O_from_python_iceking_xb133p
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,29.59 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


<IPython.core.display.Javascript object>

In [135]:
experimental_result.as_data_frame().to_csv("ggggggggggg")

<IPython.core.display.Javascript object>

In [45]:
def real_predict_h2o(models_dir_path):
    from h2o.automl import H2OAutoML
    import h2o

    h2o.init(max_mem_size="32G")

    my_model = h2o.load_model(models_dir_path)
    df = pd.read_csv("interim_data/df_completed_1_2_3_with_mv_new.csv")

    df_for_unlabeled_set = df[np.isnan(df["OFFER_STATUS"])]
    ppppp = df_for_unlabeled_set.drop(["OFFER_STATUS"], axis=1)
    x_realll = h2o.H2OFrame(ppppp)

    ggl = my_model.predict(x_realll)
    gg = ggl[0].as_data_frame().values.flatten()

    real_test = ppppp
    real_test["prediction"] = gg
    test_set_id_col = df[np.isnan(df["OFFER_STATUS"])]["TEST_SET_ID"]
    real_test = pd.concat([real_test["prediction"], test_set_id_col], axis=1)
    real_test = real_test.rename(columns={"TEST_SET_ID": "id"})

    real_test["prediction"] = real_test["prediction"].astype(int)
    real_test["id"] = real_test["id"].astype(int)
    print(ggl)

    return real_test


lllresult = real_predict_h2o(
    # Submission 5
    # models_dir_path="h2o_models_with_data/results_ts_2022-01-29_13:09:41/model_bac_0.734/GBM_1_AutoML_31_20220129_130944"
    # Submission 6
    models_dir_path="./h2o_models_with_data/results_ts_2022-01-29_18:20:43/model_withALL/StackedEnsemble_AllModels_7_AutoML_16_20220129_182045"
)

print(lllresult.prediction.value_counts())

lllresult.to_csv("predictions_versed_chimpanzee_6.csv", header=True, index=False)

lllresult

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 hours 24 mins
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.2
H2O_cluster_version_age:,3 days
H2O_cluster_name:,H2O_from_python_iceking_xb133p
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,30.88 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,p0,p1
1,0.464275,0.535725
1,0.467839,0.532161
1,0.422071,0.577929
1,0.464224,0.535776
0,0.541066,0.458934
0,0.53577,0.46423
1,0.459357,0.540643
0,0.647896,0.352104
1,0.385583,0.614417
1,0.390319,0.609681



1    2318
0    258 
Name: prediction, dtype: int64


,prediction,id
5,1,6
8,1,9
13,1,14
34,1,35
35,0,36
...,...,...
26018,1,26019
26032,0,26033
26049,1,26050
26057,1,26058


<IPython.core.display.Javascript object>